# 📚 文档问答系统完整演示

## 系统概述

这是一个基于向量检索和上下文注入的智能文档问答系统，主要技术栈包括：

- **后端**: FastAPI + LangChain + Pinecone + BGE + 千问API
- **核心功能**: 文档处理、向量检索、智能问答
- **特色**: 支持多格式文档、语义搜索、上下文对话

## 🔧 环境准备和依赖安装

In [ ]:
# 安装必要的Python包
!pip install fastapi uvicorn pydantic-settings python-dotenv \
    langchain pinecone-client sentence-transformers torch transformers \
    pypdf python-multipart dashscope numpy scikit-learn tiktoken \
    unstructured docx2txt python-docx

In [ ]:
# 导入必要的库
import os
import sys
import json
import asyncio
from datetime import datetime
from typing import List, Dict, Any, Optional

# 添加项目路径
sys.path.append('.')

print("✅ 环境准备完成")

## ⚙️ 配置环境变量

请在下方设置您的API密钥和配置参数：

In [ ]:
# 环境配置
os.environ['PINECONE_API_KEY'] = 'your_pinecone_api_key_here'
os.environ['PINECONE_ENVIRONMENT'] = 'gcp-starter'
os.environ['PINECONE_INDEX_NAME'] = 'document-qa-index'
os.environ['DASHSCOPE_API_KEY'] = 'your_dashscope_api_key_here'

print("✅ 环境变量配置完成")
print(f"Pinecone Index: {os.environ.get('PINECONE_INDEX_NAME')}")

## 📦 核心模块导入和初始化

In [ ]:
# 导入核心模块
from app.core.config import settings
from app.services.vector_store import vector_store
from app.services.embedding import encoder
from app.services.document_processor import document_processor
from app.services.qa_engine import qa_engine
from app.utils.helpers import generate_document_id, calculate_tokens

print("✅ 核心模块导入成功")
print(f"嵌入模型维度: {settings.EMBEDDING_DIMENSION}")
print(f"文档分块大小: {settings.CHUNK_SIZE}")

## 🧪 向量数据库测试

In [ ]:
# 测试向量数据库连接和索引创建
try:
    print("🔄 初始化向量数据库...")
    
    # 创建索引
    success = vector_store.create_index(recreate=False)
    
    if success:
        print("✅ 向量数据库初始化成功")
        
        # 获取统计信息
        stats = vector_store.get_stats()
        print(f"📊 索引统计: {stats}")
    else:
        print("❌ 向量数据库初始化失败")
        
except Exception as e:
    print(f"❌ 向量数据库测试失败: {str(e)}")
    print("💡 请检查Pinecone API密钥和网络连接")

## 🧠 嵌入模型测试

In [ ]:
# 测试BGE嵌入模型
try:
    print("🔄 测试嵌入模型...")
    
    # 测试文本编码
    test_texts = [
        "人工智能是计算机科学的一个分支",
        "机器学习是实现人工智能的方法",
        "深度学习是机器学习的一个子领域"
    ]
    
    # 单个文本编码
    embedding = encoder.bge_encoder.encode_single(test_texts[0])
    print(f"✅ 单文本编码成功，维度: {len(embedding)}")
    
    # 批量编码
    batch_embeddings = encoder.bge_encoder.encode_batch(test_texts)
    print(f"✅ 批量编码成功，共 {len(batch_embeddings)} 个向量")
    
    # 相似度计算
    similarity = encoder.bge_encoder.similarity(test_texts[0], test_texts[1])
    print(f"📊 文本相似度: {similarity:.4f}")
    
except Exception as e:
    print(f"❌ 嵌入模型测试失败: {str(e)}")

## 📄 文档处理测试

In [ ]:
# 创建测试文档内容
test_document_content = """
人工智能发展简史

人工智能(Artificial Intelligence, AI)是计算机科学的一个重要分支，旨在创造能够模拟人类智能行为的系统。

发展历程：
1. 1950年代：图灵测试提出，标志着AI概念的诞生
2. 1960-1970年代：专家系统兴起，符号主义AI发展
3. 1980-1990年代：机器学习算法出现，神经网络复兴
4. 2000年代至今：深度学习突破，大语言模型时代

核心技术包括：
- 机器学习(Machine Learning)
- 深度学习(Deep Learning)
- 自然语言处理(NLP)
- 计算机视觉(Computer Vision)

应用场景：
智能客服、自动驾驶、医疗诊断、金融风控、教育辅助等。
""".encode('utf-8')

print("📄 测试文档已创建")
print(f"文档大小: {len(test_document_content)} 字节")

In [ ]:
# 处理测试文档
try:
    print("🔄 处理测试文档...")
    
    processed_data = document_processor.process_document(
        test_document_content,
        "ai_history.txt",
        "text/plain"
    )
    
    print("✅ 文档处理成功")
    print(f"原始文本长度: {processed_data['raw_text_length']} 字符")
    print(f"清理后长度: {processed_data['cleaned_text_length']} 字符")
    print(f"分块数量: {processed_data['total_chunks']} 块")
    print(f"总token数: {processed_data['total_tokens']}")
    
    # 显示前几个文本块
    print("\n📝 前3个文本块预览:")
    for i, chunk in enumerate(processed_data['chunks'][:3]):
        print(f"\n块 {i+1} (tokens: {chunk['tokens']}):")
        print(f"{chunk['text'][:100]}...")
        
except Exception as e:
    print(f"❌ 文档处理失败: {str(e)}")

## 📊 向量存储测试

In [ ]:
# 将文档块存储到向量数据库
try:
    print("🔄 存储文档向量...")
    
    # 生成文档ID
    document_id = generate_document_id("ai_history.txt", test_document_content)
    print(f"文档ID: {document_id}")
    
    # 提取文本并编码
    texts = [chunk['text'] for chunk in processed_data['chunks']]
    encoded_chunks = encoder.batch_encode_with_ids(texts)
    
    # 准备向量数据
    vectors = []
    for chunk_data, encoded_chunk in zip(processed_data['chunks'], encoded_chunks):
        vector_data = {
            'id': encoded_chunk['id'],
            'values': encoded_chunk['embedding'],
            'metadata': {
                'document_id': document_id,
                'filename': "ai_history.txt",
                'chunk_index': chunk_data['chunk_index'],
                'total_chunks': chunk_data['total_chunks'],
                'text': chunk_data['text']
            }
        }
        vectors.append(vector_data)
    
    # 存储向量
    success = vector_store.upsert_vectors(vectors)
    
    if success:
        print(f"✅ 成功存储 {len(vectors)} 个向量")
        
        # 验证存储
        stats = vector_store.get_stats()
        print(f"📊 当前向量总数: {stats.get('total_vector_count', 0)}")
    else:
        print("❌ 向量存储失败")
        
except Exception as e:
    print(f"❌ 向量存储测试失败: {str(e)}")

## 🔍 语义搜索测试

In [ ]:
# 测试语义搜索功能
try:
    print("🔄 测试语义搜索...")
    
    test_queries = [
        "什么是人工智能？",
        "AI的发展历程",
        "机器学习技术"
    ]
    
    for query in test_queries:
        print(f"\n🔍 查询: '{query}'")
        
        # 执行搜索
        results = vector_store.query_vectors(
            query_vector=encoder.bge_encoder.encode_single(query).tolist(),
            top_k=3
        )
        
        print(f"找到 {len(results)} 个相关结果:")
        for i, result in enumerate(results, 1):
            print(f"  {i}. 相似度: {result['score']:.4f}")
            print(f"     内容: {result['metadata']['text'][:80]}...")
            
except Exception as e:
    print(f"❌ 语义搜索测试失败: {str(e)}")

## 💬 智能问答测试

In [ ]:
# 测试问答功能
try:
    print("🔄 测试智能问答...")
    
    test_questions = [
        "人工智能是什么？",
        "AI有哪些核心技术？",
        "人工智能的应用场景有哪些？"
    ]
    
    for question in test_questions:
        print(f"\n❓ 问题: {question}")
        
        # 执行问答
        result = asyncio.run(qa_engine.answer_question(question))
        
        print(f"🤖 回答: {result['answer']}")
        print(f"📈 置信度: {result['confidence']:.2f}")
        
        if result['sources']:
            print("📎 引用来源:")
            for source in result['sources'][:2]:  # 只显示前2个来源
                print(f"  - {source['filename']}: 相似度 {source['score']:.3f}")
        
        print("-" * 50)
        
except Exception as e:
    print(f"❌ 智能问答测试失败: {str(e)}")

## 📊 性能基准测试

In [ ]:
import time

# 性能测试
def benchmark_function(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    return result, end_time - start_time

print("⏱️ 性能基准测试")

# 测试嵌入编码性能
test_text = "这是一个性能测试的示例文本，用来评估编码速度。" * 10
_, encoding_time = benchmark_function(encoder.bge_encoder.encode_single, test_text)
print(f"嵌入编码耗时: {encoding_time:.4f}秒")

# 测试搜索性能
query_vector = encoder.bge_encoder.encode_single("测试查询").tolist()
_, search_time = benchmark_function(vector_store.query_vectors, query_vector, 5)
print(f"向量搜索耗时: {search_time:.4f}秒")

## 🛠️ 系统维护工具

In [ ]:
# 系统信息和维护
def get_system_info():
    info = {
        "timestamp": datetime.now().isoformat(),
        "config": {
            "embedding_dimension": settings.EMBEDDING_DIMENSION,
            "chunk_size": settings.CHUNK_SIZE,
            "chunk_overlap": settings.CHUNK_OVERLAP,
            "pinecone_index": settings.PINECONE_INDEX_NAME
        }
    }
    
    try:
        info["vector_stats"] = vector_store.get_stats()
    except:
        info["vector_stats"] = "无法获取向量统计信息"
    
    return info

# 显示系统信息
system_info = get_system_info()
print("🔧 系统信息:")
print(json.dumps(system_info, indent=2, ensure_ascii=False))

In [ ]:
# 清理测试数据（可选）
def cleanup_test_data():
    print("🧹 清理测试数据...")
    # 这里可以添加清理逻辑
    print("✅ 清理完成")

# cleanup_test_data()  # 取消注释以执行清理

## 📈 测试总结和下一步

In [ ]:
print("🎉 文档问答系统测试完成！")
print("\n📋 测试结果概览:")
print("✅ 向量数据库连接正常")
print("✅ 嵌入模型工作正常")
print("✅ 文档处理功能正常")
print("✅ 语义搜索功能正常")
print("✅ 智能问答功能正常")

print("\n🚀 下一步建议:")
print("1. 上传真实文档进行测试")
print("2. 调整分块参数优化效果")
print("3. 集成前端界面")
print("4. 部署到生产环境")

## 📚 参考资料和API文档

- [FastAPI官方文档](https://fastapi.tiangolo.com/)
- [Pinecone向量数据库](https://www.pinecone.io/)
- [LangChain文档](https://docs.langchain.com/)
- [千问API文档](https://help.aliyun.com/zh/dashscope/)
- [Sentence Transformers](https://www.sbert.net/)